In [1]:
import ast
import os
from pathlib import Path
from pprint import pprint
from typing import List, Tuple

import numpy as np
import padre_meddea
import pandas as pd
from astropy.io import fits
from astropy.time import Time
from padre_meddea import config
from padre_meddea.calibration.calibration import process_file
from padre_meddea.io.file_tools import (
    read_fits_l0_event_list,
    read_fits_l0_housekeeping,
    read_fits_l0_spectrum,
)
from padre_meddea.io.fits_tools import (
    CUSTOM_ATTRS_PATH,
    _concatenate_input_files,
    _init_hdul_structure,
    _sort_hdul_template,
    _write_output_file,
    get_hdu_data_times,
)
from padre_meddea.util.util import (
    calc_time,
    create_science_filename,
    parse_science_filename,
)
from solarnet_metadata.schema import SOLARNETSchema
from solarnet_metadata.validation import validate_file, validate_header

In [2]:
def get_file_header_times(file_path: Path) -> Tuple[Time, Time]:
    """
    Get the DATE-BEG and DATE-END from the FITS file header.

    Parameters
    ----------
    file_path : Path
        Path to the FITS file.

    Returns
    -------
    Tuple[Time, Time]
        A tuple containing the start date (DATE-BEG) and end date (DATE-END) as astropy Time objects.

    Raises
    -------
    ValueError
        If the file does not contain DATE-BEG, DATE-END, or DATEREF keywords.
    """
    hdul = fits.open(file_path)
    header = hdul[0].header.copy()
    hdul.close()

    # Get Start Date
    if "DATE-BEG" in header:
        date_beg = Time(header["DATE-BEG"])
    elif "DATEREF" in header:
        date_beg = Time(header["DATEREF"])
    else:
        raise ValueError(f"File {file_path} does not contain DATE-BEG or DATEREF.")

    # Get End Date
    if "DATE-END" in header:
        date_end = Time(header["DATE-END"])
    elif "DATEREF" in header:
        date_end = Time(header["DATEREF"])
    else:
        raise ValueError(f"File {file_path} does not contain DATE-END or DATEREF.")

    return date_beg, date_end


def get_file_data_times(file_path: Path) -> Time:
    """
    Extract time information from the data within a FITS file.

    This function parses times differently based on the file descriptor (eventlist, hk, spec)
    extracted from the filename. It accesses the appropriate HDU and data columns for
    each file type to calculate accurate time values.

    Parameters
    ----------
    file_path : Path
        Path to the FITS file to extract time data from

    Returns
    -------
    Time
        Astropy Time object containing the time values extracted from the file data

    Raises
    ------
    ValueError
        If the file descriptor is not recognized or supported
    """
    # Get the File Desctiptor
    # We need to parse times differently for Photon / Spectrum / HK
    file_meta = parse_science_filename(file_path)
    file_descriptor = file_meta["descriptor"]
    times = None

    hdul = fits.open(file_path)
    # Calculate Times based on the file descriptor
    if file_descriptor == "photon":
        times = calc_time(
            hdul["SCI"].data["pkttimes"],
            hdul["SCI"].data["pktclock"],
            hdul["SCI"].data["clocks"],
        )
    elif file_descriptor == "housekeeping":
        times = calc_time(hdul["HK"].data["timestamp"])
    elif file_descriptor == "spectrum":
        times = calc_time(hdul["PKT"].data["pkttimes"], hdul["PKT"].data["pktclock"])
    else:
        raise ValueError(f"File contents of {file_path} not recogized.")
    # Explicitly Open and Close File - Windows Garbage Disposer cannot be trusted.
    hdul.close()

    return times

## Re-Process RAW Data

In [3]:
# NOTE: CHANGE ME TO YOUR OWN DATASET ROOT
dataset_root = Path("/Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets")

# Path to Raw .DAT Files
dataset_dir = dataset_root / "050425"
raw_files = [f for f in dataset_dir.glob("PADREMD*.DAT") if "Merged" not in f.name]
print(f"Found {len(raw_files)} raw files.")
raw_files[:2]

Found 36 raw files.


[PosixPath('/Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT'),
 PosixPath('/Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504131124.DAT')]

In [4]:
processed_dir = dataset_root / "050425_processed"
processed_dir.mkdir(exist_ok=True)

# We can't specify the the output directory for the process_file function,
# so we need to change the current working directory to the processed directory.
current_dir = os.getcwd()
os.chdir(processed_dir)

failed_files = []
# Process each raw file
for raw_file in raw_files:
    print(f"Processing file: {raw_file.name}")
    
    try:
        process_file(raw_file, overwrite=True)
    except ValueError as e:
        print(f"Error processing {raw_file.name}: {e}")
        failed_files
    
# Change back to the original directory
os.chdir(current_dir)

# List the processed files
processed_files = list(processed_dir.glob("*.fits"))
print(f"Processed {len(processed_files)} files.")
    

2025-06-23 14:09:01 - swxsoc - INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT.


Processing file: PADREMDA0_250504095122.DAT
INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT. [padre_meddea.calibration.calibration]


2025-06-23 14:09:02 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT : UserWarning: Sequence count are out of order.
2025-06-23 14:09:02 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT : ChecksumWarning: Packet 548 has a checksum error.
2025-06-23 14:09:02 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT : ChecksumWarning: Packet 549 has a checksum error.
2025-06-23 14:09:02 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA0_250504095122.DAT : ChecksumWarning: Packet 7047 has a checksum error.
2025-06-23 14:09:02 - swxsoc - WARNING: Validation Finding for File : /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMD

INFO: PADREMDA0_250504095122.DAT: Found photon data [padre_meddea.io.file_tools]


/opt/homebrew/Caskroom/miniforge/base/envs/swxsoc/lib/python3.12/site-packages/ccsdspy/packet_types.py:364: UserWarning: Sequence count are out of order.
  warnings.warn("Sequence count are out of order.", UserWarning)
2025-06-23 14:09:02 - swxsoc - INFO: PADREMDA0_250504095122.DAT: Found baseline measurements in photon data.


...

Processing file: PADREMDA2_250504081536.DAT
INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDA2_250504081536.DAT. [padre_meddea.calibration.calibration]
INFO: PADREMDA2_250504081536.DAT: Found spectrum data [padre_meddea.io.file_tools]


/opt/homebrew/Caskroom/miniforge/base/envs/swxsoc/lib/python3.12/site-packages/ccsdspy/packet_types.py:361: UserWarning: Missing packets found [869, 871, 1117].
  warnings.warn(f"Missing packets found {missing_elements}.", UserWarning)
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-06-23 14:11:47 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-06-23 14:11:47 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-06-23 14:11:47 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Timestream: Unable to locate credentials [swxsoc.util.util]
2025-06-23 14:11:47 - swxsoc - ERROR: Failed to write to Timestream: Unable to locate credentials
ERROR: Failed to write to Ti

Processing file: PADREMDU8_250504153121.DAT
INFO: Processing file /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425/PADREMDU8_250504153121.DAT. [padre_meddea.calibration.calibration]
INFO: PADREMDU8_250504153121.DAT: Found housekeeping data [padre_meddea.housekeeping.housekeeping]


2025-06-23 14:11:51 - swxsoc - WARNING: Version 3.0.1.dev39+g2b91f4b not found in Tags for https://github.com/IHDE-Alliance/solarnet_metadata.git. Using HEAD.
2025-06-23 14:11:52 - astropy - WARNING: VerifyWarning: Card is too long, comment will be truncated.


Processed 33 files.


## Check for SOLARNET Compliance in the L0 Data

In [5]:
# Create Custome PADRE SOLARNET schema
padre_schema = SOLARNETSchema(schema_layers=[CUSTOM_ATTRS_PATH])

files = []
all_findings = []
for processed_file in processed_files:
    # Validate the firt Processed File against the SOALRNET schema
    file_findings = validate_file(
        file_path=processed_file,
        warn_empty_keyword=True,
        warn_no_comment=False,
        warn_data_type=True,
        schema=padre_schema,
    )
    all_findings.extend(file_findings)
    files.extend([processed_file.name] * len(file_findings))

In [6]:
df = pd.DataFrame([files, all_findings]).T
df.columns = ["file", "findings"]

# Group by findings and get unique filenames for each finding
findings_summary = df.groupby('findings')['file'].unique().reset_index()

# Optionally, add a count of files for each finding
findings_summary['file_count'] = findings_summary['file'].apply(len)

# Sort by most common findings first
findings_summary = findings_summary.sort_values('file_count', ascending=True)

findings_summary.head(10)

,findings,file,file_count
13,Observation Header 2: Keyword 'TCUNI1' not fou...,[padre_meddea_l0test_housekeeping_20250504T055...,1
25,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T153...,1
24,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T153...,1
18,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T153...,1
11,Observation Header 2: Keyword 'JDREF' not foun...,[padre_meddea_l0test_housekeeping_20250504T055...,1
16,Observation Header 2: Keyword 'TUNIT1' not fou...,[padre_meddea_l0test_housekeeping_20250504T055...,1
26,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T153...,1
15,Observation Header 2: Keyword 'TREFPOS' not fo...,[padre_meddea_l0test_housekeeping_20250504T055...,1
2,Observation Header 1: Keyword 'TFIELDS' not fo...,[padre_meddea_l0test_photon_20250504T153114_v0...,25
23,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_photon_20250504T153114_v0...,33


In [7]:
pprint(findings_summary["findings"][:10].values)

array(["Observation Header 2: Keyword 'TCUNI1' not found in the schema. Cannot Validate Data Type.",
       'Observation Header 2: Missing Required Attribute: DATE-BEG',
       'Observation Header 2: Missing Required Attribute: DATASUM',
       'Observation Header 2: Missing Required Attribute: CHECKSUM',
       "Observation Header 2: Keyword 'JDREF' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 2: Keyword 'TUNIT1' not found in the schema. Cannot Validate Data Type.",
       'Observation Header 2: Missing Required Attribute: DATEREF',
       "Observation Header 2: Keyword 'TREFPOS' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 1: Keyword 'TFIELDS' not found in the schema. Cannot Validate Data Type.",
       'Observation Header 2: Missing Required Attribute: CUNITia. No pattern match for CUNITia with pattern CUNIT(?P<i>[1-9])(?P<a>[A-Z])?'],
      dtype=object)


## Prepare L0 Data for Data Reduction

In [8]:
test_files = list(processed_dir.rglob("*.fits"))
file_meta = []
for filename in test_files:

    meta = parse_science_filename(filename.name)
    meta["path"] = filename

    # Get the date-beg and date-end from the header
    date_beg, date_end = get_file_header_times(filename)
    date_beg_iso = date_beg.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    date_end_iso = date_end.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    meta["date_beg"] = date_beg_iso
    meta["date_end"] = date_end_iso

    # Get the File Data Times
    times: Time = get_file_data_times(filename)
    meta["time_beg"] = times[0].iso
    meta["time_end"] = times[-1].iso

    file_meta.append(meta)

df = pd.DataFrame(file_meta).sort_values(by=["date_beg", "time_beg"])
df.head()

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
6,meddea,None,True,2000-01-01T00:00:00.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 09:12:57.324
2,meddea,None,True,2025-05-04T10:38:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 11:49:11.406
9,meddea,None,True,2025-05-04T11:49:21.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 13:00:31.420
13,meddea,None,True,2025-05-04T14:12:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 15:23:21.447
15,meddea,None,True,2025-05-04T07:04:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 08:15:11.363


In [9]:
df["descriptor"].value_counts()

descriptor
photon          23
spectrum         8
housekeeping     2
Name: count, dtype: int64

# Split & Modify Raw Data for Testing

**Cases to be tested:**
1. (base case) Concatenating a series of files within the same day that may be of order, but completely disjoint.
2. Concatenating a series of files that span multiple days, testing that we can handle the day change and generate multiple output files.
3. Concatenating a series of files within the same day, but inserting a an overlapping file that is out-of-order into an existing file that has a missing time range. 

So I think we'll need 4 files to test for each data type:
- 3 files `[A, B, C]` that are disjoint within the same day
- 1 file `[D]` that is disjoint across multiple days

Test cases
1. `A`, `B`, `C` are disjoint within the same day
2. `A`, `B`, `D` are disjoint across multiple days
3. Concatenate `A`, `C` and then insert `B` which is out-of-order and overlaps with `AC`.

In [10]:
stripped_dir = dataset_root / "050425_stripped"
stripped_dir.mkdir(exist_ok=True)

In [11]:
def strip_fits_file(input_path, output_path, max_rows=10):
    with fits.open(input_path, mode='readonly') as hdul:
        stripped_hdul = fits.HDUList()
        stripped_hdul.append(hdul[0].copy())  # Copy primary HDU fully

        for hdu in hdul[1:]:
            if isinstance(hdu, fits.BinTableHDU):
                print(f"Processing BinTable: {hdu.name}, Data Type: {hdu.header['BTYPE']} Shape: {hdu.data.shape}")
                n_rows = min(len(hdu.data), max_rows)
                new_data = hdu.data[:n_rows].copy()
                new_hdu = fits.BinTableHDU(data=new_data, header=hdu.header.copy())
                new_hdu.name = hdu.name  # Preserve HDU name
                stripped_hdul.append(new_hdu)
            elif isinstance(hdu, fits.TableHDU):
                print(f"Processing Table: {hdu.name}, Data Type: {hdu.header['BTYPE']} Shape: {hdu.data.shape}")
                # For Table HDUs, we can also limit the number of rows
                n_rows = min(len(hdu.data), max_rows)
                new_data = hdu.data[:n_rows].copy()
                new_hdu = fits.TableHDU(data=new_data, header=hdu.header.copy())
                new_hdu.name = hdu.name
                stripped_hdul.append(new_hdu)
            elif isinstance(hdu, fits.ImageHDU):
                print(f"Processing ImageHDU: {hdu.name}, Data Type: {hdu.header['BTYPE']} Shape: {hdu.data.shape}")
                # For Image HDUs, limit the data to the first max_rows rows
                n_rows = min(hdu.data.shape[0], max_rows) if hdu.data.ndim > 0 else 1
                new_data = hdu.data[:n_rows] if hdu.data.ndim > 0 else hdu.data
                # Create a new ImageHDU with the limited data
                new_hdu = fits.CompImageHDU(data=new_data, header=hdu.header.copy())
                new_hdu.name = hdu.name  # Preserve HDU name
                stripped_hdul.append(new_hdu)
                print(f"Compressed Image HDU: {new_hdu.name}, Data Type: {new_hdu.header['BTYPE']} Shape: {new_hdu.data.shape} ")

        stripped_hdul.writeto(output_path, overwrite=True)
        print(f"Stripped FITS file written to: {output_path}")

    
def strip_fits_file_multiday(input_path, output_path, max_rows=10):
    with fits.open(input_path, mode="readonly") as hdul:
        stripped_hdul = fits.HDUList()
        stripped_hdul.append(hdul[0].copy())  # Copy primary HDU fully

        for hdu in hdul[1:]:
            if isinstance(hdu, fits.BinTableHDU):
                n_rows = min(len(hdu.data), max_rows)
                new_data = hdu.data[:n_rows].copy()

                split_idx = int(n_rows / 2)
                # Make sure the Data Spans Multiple Days
                if hdu.name == "SCI" and all(
                    col in new_data.names for col in ["pkttimes", "pktclock", "clocks"]
                ):
                    # Make the last row's time be one day later (86400 seconds = 1 day)
                    new_data["pkttimes"][split_idx:] = (
                        new_data["pkttimes"][split_idx:] + 86400
                    )

                    # Adjust pktclock and clocks proportionally if needed
                    if n_rows > 1:
                        original_pktclock = new_data["pktclock"][0]
                        original_clocks = new_data["clocks"][0]

                        # Calculate the time step between rows 0 and 1 for scaling
                        pkt_step = new_data["pktclock"][1] - original_pktclock
                        clock_step = new_data["clocks"][1] - original_clocks

                        # Scale to make last row cross to next day
                        new_data["pktclock"][-1] = original_pktclock + pkt_step * (
                            86400 / 0.004
                        )
                        new_data["clocks"][-1] = original_clocks + clock_step * (
                            86400 / 0.004
                        )

                elif hdu.name == "HK" and "timestamp" in new_data.names:
                    # For housekeeping files
                    new_data["timestamp"][split_idx:] = (
                        new_data["timestamp"][split_idx:] + 86400
                    )

                elif hdu.name == "PKT" and "pkttimes" in new_data.names:
                    # For spectrum files
                    new_data["pkttimes"][split_idx:] = (
                        new_data["pkttimes"][split_idx:] + 86400
                    )
                    if "pktclock" in new_data.names and n_rows > 1:
                        pkt_step = new_data["pktclock"][1] - new_data["pktclock"][0]
                        new_data["pktclock"][-1] = new_data["pktclock"][
                            0
                        ] + pkt_step * (86400 / 0.004)

                new_hdu = fits.BinTableHDU(data=new_data, header=hdu.header.copy())
                new_hdu.name = hdu.name  # Preserve HDU name
                stripped_hdul.append(new_hdu)
            elif isinstance(hdu, fits.ImageHDU):
                # For Image HDUs, limit the data to the first max_rows rows
                n_rows = min(hdu.data.shape[0], max_rows) if hdu.data.ndim > 0 else 1
                new_data = hdu.data[:n_rows] if hdu.data.ndim > 0 else hdu.data
                # Create a new ImageHDU with the limited data
                new_hdu = fits.CompImageHDU(data=new_data, header=hdu.header.copy())
                new_hdu.name = hdu.name  # Preserve HDU name
                stripped_hdul.append(new_hdu)

        # Write the modified file
        stripped_hdul.writeto(output_path, overwrite=True)

        print(f"Stripped FITS file written to: {output_path}")

    return output_path, new_hdu


def update_stripped_metadata(output_path):
    # Get the new times and update the primary header
    new_times = get_file_data_times(output_path)
    
    with fits.open(output_path, mode="update") as hdul:
        for hdu in hdul:
            hdu.header["DATE-BEG"] = new_times[0].fits
            hdu.header["DATE-END"] = new_times[-1].fits
            if "DATE-AVG" in hdu.header:
                hdu.header["DATE-AVG"] = Time(
                    (new_times[0].mjd + new_times[-1].mjd) / 2, format="mjd"
                ).fits
            if "DATEREF" in hdu.header:
                hdu.header["DATEREF"] = new_times[0].fits

## Photon Data (Event List)

lets do photon data first since we have the most of it


In [12]:
photon_files = df[df["descriptor"] == "photon"][1:5].reset_index(drop=True)

# 4 Photon Files
photon_files

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
0,meddea,None,True,2025-05-04T05:53:11.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:53:11.353,2025-05-04 07:37:49.472
1,meddea,None,True,2025-05-04T07:37:49.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 07:37:49.472,2025-05-04 08:03:30.385
2,meddea,None,True,2025-05-04T08:03:30.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 08:03:30.385,2025-05-04 08:18:30.919
3,meddea,None,True,2025-05-04T08:32:34.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 08:32:34.299,2025-05-04 08:46:05.582


In [13]:
with fits.open(photon_files["path"][0], mode="readonly") as hdul:
    print(hdul.info())

Filename: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_processed/padre_meddea_l0test_photon_20250504T055311_v0.1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   ()      
  1  SCI           1 BinTableHDU     57   1264891R x 9C   [I, I, B, B, I, I, J, J, K]   
  2  PKT           1 BinTableHDU     65   16669R x 12C   [I, J, J, I, I, I, I, I, I, J, D, D]   
None


In [14]:
num_photon_files = len(photon_files)
for index, row in photon_files.iterrows():
    input_path = row["path"]
    output_path = stripped_dir / input_path.name
    
    # First 3 files are stripped normally
    if index < num_photon_files - 1:
        # Strip the file normally
        strip_fits_file(input_path, output_path, max_rows=10)
    # Last file, we need to ensure it spans multiple days
    else:
        # Strip the file with multi-day handling
        strip_fits_file_multiday(input_path, output_path, max_rows=10)
    # Update the metadata in the stripped file
    update_stripped_metadata(output_path)

Processing BinTable: SCI, Data Type: photon Shape: (1264891,)
Processing BinTable: PKT, Data Type: photon Shape: (16669,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T055311_v0.1.0.fits
Processing BinTable: SCI, Data Type: photon Shape: (1285553,)
Processing BinTable: PKT, Data Type: photon Shape: (9199,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T073749_v0.1.0.fits
Processing BinTable: SCI, Data Type: photon Shape: (1286220,)
Processing BinTable: PKT, Data Type: photon Shape: (8962,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T080330_v0.1.0.fits
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T083234_v0.1

/var/folders/5l/_5r0pdg15fxg1_rkgmd3c1dm0000gn/T/ipykernel_45807/3498688488.py:87: RuntimeWarning: invalid value encountered in cast
  new_data["pktclock"][-1] = new_data["pktclock"][


In [15]:
stripped_photon_files = list(stripped_dir.glob("*photon*.fits"))
file_meta = []
for filename in stripped_photon_files:
    meta = parse_science_filename(filename.name)
    meta["path"] = filename

    # Get the date-beg and date-end from the header
    date_beg, date_end = get_file_header_times(filename)
    date_beg_iso = date_beg.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    date_end_iso = date_end.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    meta["date_beg"] = date_beg_iso
    meta["date_end"] = date_end_iso

    # Get the File Data Times
    times: Time = get_file_data_times(filename)
    meta["time_beg"] = times[0].iso
    meta["time_end"] = times[-1].iso

    file_meta.append(meta)

ph_df = pd.DataFrame(file_meta).sort_values(by=["date_beg", "time_beg"])
ph_df.head()

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
3,meddea,None,True,2025-05-04T05:53:11.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:53:11.353,2025-05-04 05:53:11.830
0,meddea,None,True,2025-05-04T07:37:49.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 07:37:49.472,2025-05-04 07:37:49.476
2,meddea,None,True,2025-05-04T08:03:30.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 08:03:30.385,2025-05-04 08:03:30.390
1,meddea,None,True,2025-05-04T08:32:34.000,l0,0.1.0,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-05,2025-05-04 08:32:34.299,2025-05-05 08:32:34.430


In [16]:
for path in ph_df["path"]:
    with fits.open(path) as hdul:
        print(hdul.info())

Filename: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T055311_v0.1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   ()      
  1  SCI           1 BinTableHDU     57   10R x 9C   [I, I, B, B, I, I, J, J, K]   
  2  PKT           1 BinTableHDU     66   10R x 12C   [I, J, J, I, I, I, I, I, I, J, D, D]   
None
Filename: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T073749_v0.1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   ()      
  1  SCI           1 BinTableHDU     57   10R x 9C   [I, I, B, B, I, I, J, J, K]   
  2  PKT           1 BinTableHDU     66   10R x 12C   [I, J, J, I, I, I, I, I, I, J, D, D]   
None
Filename: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_photon_20250504T080330_

## Spectrum Files

In [17]:
df[df["descriptor"] == "spectrum"]

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
2,meddea,None,True,2025-05-04T10:38:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 11:49:11.406
9,meddea,None,True,2025-05-04T11:49:21.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 13:00:31.420
13,meddea,None,True,2025-05-04T14:12:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 15:23:21.447
15,meddea,None,True,2025-05-04T07:04:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 08:15:11.363
16,meddea,None,True,2025-05-04T08:15:21.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 09:26:41.378
20,meddea,None,True,2025-05-04T15:23:31.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 15:31:01.449
25,meddea,None,True,2025-05-04T13:00:41.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 14:12:01.433
19,meddea,None,True,2025-05-04T15:31:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 15:31:11.449,2025-05-04 15:33:09.809


In [18]:
spectrum_files = df[df["descriptor"] == "spectrum"][:4].reset_index(drop=True)

# 4 Spectrum Files
spectrum_files

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
0,meddea,None,True,2025-05-04T10:38:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 11:49:11.406
1,meddea,None,True,2025-05-04T11:49:21.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 13:00:31.420
2,meddea,None,True,2025-05-04T14:12:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 15:23:21.447
3,meddea,None,True,2025-05-04T07:04:11.000,l0,0.1.0,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 08:15:11.363


In [19]:
with fits.open(spectrum_files["path"][0], mode="readonly") as hdul:
    print(hdul.info())

Filename: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_processed/padre_meddea_l0test_spectrum_20250504T103811_v0.1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   ()      
  1  SPEC          1 ImageHDU        30   (512, 24, 426)   float64   
  2  PKT           1 BinTableHDU     47   426R x 5C   [J, J, 24I, 24I, I]   
None


In [20]:
num_spectrum_files = len(spectrum_files)
for index, row in spectrum_files.iterrows():
    input_path = row["path"]
    output_path = stripped_dir / input_path.name
    
    # First 3 files are stripped normally
    if index < num_spectrum_files - 1:
        # Strip the file normally
        strip_fits_file(input_path, output_path, max_rows=10)
    # Last file, we need to ensure it spans multiple days
    else:
        # Strip the file with multi-day handling
        strip_fits_file_multiday(input_path, output_path, max_rows=10)
    # Update the metadata in the stripped file
    update_stripped_metadata(output_path)

Processing ImageHDU: SPEC, Data Type: spectrum Shape: (426, 24, 512)
Compressed Image HDU: SPEC, Data Type: spectrum Shape: (10, 24, 512) 
Processing BinTable: PKT, Data Type: spectrum Shape: (426,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_spectrum_20250504T103811_v0.1.0.fits
Processing ImageHDU: SPEC, Data Type: spectrum Shape: (426, 24, 512)
Compressed Image HDU: SPEC, Data Type: spectrum Shape: (10, 24, 512) 
Processing BinTable: PKT, Data Type: spectrum Shape: (426,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_spectrum_20250504T114921_v0.1.0.fits
Processing ImageHDU: SPEC, Data Type: spectrum Shape: (426, 24, 512)
Compressed Image HDU: SPEC, Data Type: spectrum Shape: (10, 24, 512) 
Processing BinTable: PKT, Data Type: spectrum Shape: (426,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padr

/var/folders/5l/_5r0pdg15fxg1_rkgmd3c1dm0000gn/T/ipykernel_45807/3498688488.py:87: RuntimeWarning: invalid value encountered in cast
  new_data["pktclock"][-1] = new_data["pktclock"][


In [21]:
stripped_spec_files = list(stripped_dir.glob("*spec*.fits"))
file_meta = []
for filename in stripped_spec_files:
    meta = parse_science_filename(filename.name)
    meta["path"] = filename.name

    # Get the date-beg and date-end from the header
    date_beg, date_end = get_file_header_times(filename)
    date_beg_iso = date_beg.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    date_end_iso = date_end.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    meta["date_beg"] = date_beg_iso
    meta["date_end"] = date_end_iso

    # Get the File Data Times
    times: Time = get_file_data_times(filename)
    meta["time_beg"] = times[0].iso
    meta["time_end"] = times[-1].iso
    
    file_meta.append(meta)

spec_df = pd.DataFrame(file_meta).sort_values(by=["date_beg", "time_beg"])
spec_df.head()

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
0,meddea,None,True,2025-05-04T10:38:11.000,l0,0.1.0,spectrum,padre_meddea_l0test_spectrum_20250504T103811_v...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 10:39:51.392
1,meddea,None,True,2025-05-04T11:49:21.000,l0,0.1.0,spectrum,padre_meddea_l0test_spectrum_20250504T114921_v...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 11:50:51.406
2,meddea,None,True,2025-05-04T14:12:11.000,l0,0.1.0,spectrum,padre_meddea_l0test_spectrum_20250504T141211_v...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 14:13:41.434
3,meddea,None,True,2025-05-04T07:04:11.000,l0,0.1.0,spectrum,padre_meddea_l0test_spectrum_20250504T070411_v...,2000-01-01,2025-05-05,2000-01-01 00:00:00.000,2025-05-05 07:09:15.748


In [22]:
spec_df["path"].values

array(['padre_meddea_l0test_spectrum_20250504T103811_v0.1.0.fits',
       'padre_meddea_l0test_spectrum_20250504T114921_v0.1.0.fits',
       'padre_meddea_l0test_spectrum_20250504T141211_v0.1.0.fits',
       'padre_meddea_l0test_spectrum_20250504T070411_v0.1.0.fits'],
      dtype=object)

In [23]:
get_file_data_times(stripped_dir / spec_df["path"].values[3])

<Time object: scale='utc' format='iso' value=['2000-01-01 00:00:00.000' '2025-05-04 07:04:21.349'
 '2025-05-04 07:04:31.349' '2025-05-04 07:04:41.349'
 '2025-05-04 07:04:51.349' '2025-05-05 07:05:01.349'
 '2025-05-05 07:05:11.349' '2025-05-05 07:05:21.349'
 '2025-05-05 07:05:31.349' '2025-05-05 07:09:15.748']>

## HouseKeeping Files

In [24]:
hk_files = df[df["descriptor"] == "housekeeping"].reset_index(drop=True)

# 1 Spectrum File
hk_files

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
0,meddea,None,True,2025-05-04T05:51:38.000,l0,0.1.0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:51:38.000,2025-05-04 15:31:08.000
1,meddea,None,True,2025-05-04T15:31:18.000,l0,0.1.0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 15:31:18.000,2025-05-04 15:34:08.000


In [25]:
orig_hk_path = hk_files["path"].iloc[0]

# 1. Strip the original file
base_filename = orig_hk_path.name
output_path = stripped_dir / base_filename
strip_fits_file(orig_hk_path, output_path, max_rows=10)
update_stripped_metadata(output_path)

# Get the time info to create sequential files
with fits.open(output_path) as hdul:
    end_time_str = hdul[0].header.get('DATE-END')
end_time = Time(end_time_str)

# 2. Create 3 sequential duplicates
duplicate_files = []
current_start_time = end_time
for i in range(1, 4):
    # Create new filename with incremented timestamp
    new_time_str = current_start_time.isot.replace('-', '').replace(':', '')[:15]
    new_filename = f"padre_meddea_l0test_housekeeping_{new_time_str}_v0.1.0.fits"
    new_path = stripped_dir / new_filename
    
    # For the last file, use multiday function
    if i == 3:
        # 3. For the last duplicate, make it span multiple days
        print(f"Creating multiday duplicate {i}: {new_filename}")
        strip_fits_file_multiday(output_path, new_path, max_rows=10)
    else:
        # Normal strip for first two duplicates
        strip_fits_file(output_path, new_path, max_rows=10)
    
    # Now adjust the timestamps in the file
    with fits.open(new_path, mode="update") as hdul:
        for hdu in hdul:
            # Update time fields in each HDU
            if isinstance(hdu, fits.BinTableHDU) and "timestamp" in hdu.data.names:
                # For regular files, advance time by 2 minutes (120 seconds)
                time_offset = 120 * i
                if time_offset > 0:
                    hdu.data["timestamp"] += time_offset
    
    # Update metadata based on new content
    update_stripped_metadata(new_path)
    
    # Get the end time for the next file
    with fits.open(new_path) as hdul:
        new_end_time_str = hdul[0].header.get('DATE-END')
    current_start_time = Time(new_end_time_str)
    
    duplicate_files.append(new_path)
    print(f"Created duplicate {i}: {new_path}")

print(f"\nCreated original stripped file and {len(duplicate_files)} duplicates:")
print(f"Original: {output_path}")
for i, file in enumerate(duplicate_files):
    print(f"Duplicate {i+1}: {file}")

Processing BinTable: HK, Data Type: housekeeping Shape: (3478,)
Processing BinTable: READ, Data Type: housekeeping Shape: (1,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_housekeeping_20250504T055138_v0.1.0.fits
Processing BinTable: HK, Data Type: housekeeping Shape: (10,)
Processing BinTable: READ, Data Type: housekeeping Shape: (1,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_housekeeping_20250504T055308_v0.1.0.fits
Created duplicate 1: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_housekeeping_20250504T055308_v0.1.0.fits
Processing BinTable: HK, Data Type: housekeeping Shape: (10,)
Processing BinTable: READ, Data Type: housekeeping Shape: (1,)
Stripped FITS file written to: /Users/andrewrobbertz/__SOC_CODE__/_data_/padre-l0-packets/050425_stripped/padre_meddea_l0test_housek

In [26]:
stripped_hk_files = list(stripped_dir.glob("*housekeeping*.fits"))
file_meta = []
for filename in stripped_hk_files:
    meta = parse_science_filename(filename.name)
    meta["path"] = filename.name

    # Get the date-beg and date-end from the header
    date_beg, date_end = get_file_header_times(filename)
    date_beg_iso = date_beg.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    date_end_iso = date_end.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    meta["date_beg"] = date_beg_iso
    meta["date_end"] = date_end_iso

    # Get the File Data Times
    times: Time = get_file_data_times(filename)
    meta["time_beg"] = times[0].iso
    meta["time_end"] = times[-1].iso

    file_meta.append(meta)

hk_df = pd.DataFrame(file_meta).sort_values(by=["date_beg", "time_beg"])
hk_df.head()

,instrument,mode,test,time,level,version,descriptor,path,date_beg,date_end,time_beg,time_end
1,meddea,None,True,2025-05-04T05:51:38.000,l0,0.1.0,housekeeping,padre_meddea_l0test_housekeeping_20250504T0551...,2025-05-04,2025-05-04,2025-05-04 05:51:38.000,2025-05-04 05:53:08.000
2,meddea,None,True,2025-05-04T05:53:08.000,l0,0.1.0,housekeeping,padre_meddea_l0test_housekeeping_20250504T0553...,2025-05-04,2025-05-04,2025-05-04 05:53:38.000,2025-05-04 05:55:08.000
0,meddea,None,True,2025-05-04T05:55:08.000,l0,0.1.0,housekeeping,padre_meddea_l0test_housekeeping_20250504T0555...,2025-05-04,2025-05-04,2025-05-04 05:55:38.000,2025-05-04 05:57:08.000
3,meddea,None,True,2025-05-04T05:57:08.000,l0,0.1.0,housekeeping,padre_meddea_l0test_housekeeping_20250504T0557...,2025-05-04,2025-05-05,2025-05-04 05:57:38.000,2025-05-05 05:59:08.000


# Review Results

In [27]:
stripped_files = list(stripped_dir.glob("*.fits"))
file_meta = []
for filename in stripped_files:
    meta = parse_science_filename(filename.name)
    meta["path"] = filename

    # Get the date-beg and date-end from the header
    date_beg, date_end = get_file_header_times(filename)
    date_beg_iso = date_beg.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    date_end_iso = date_end.iso[0:10]  # Convert to ISO format YYYY-MM-DD
    meta["date_beg"] = date_beg_iso
    meta["date_end"] = date_end_iso

    # Get the File Data Times
    times: Time = get_file_data_times(filename)
    meta["time_beg"] = times[0].iso
    meta["time_end"] = times[-1].iso

    file_meta.append(meta)

result_df = pd.DataFrame(file_meta).sort_values(by=["descriptor", "time_beg"])
result_df.drop(columns=["instrument", "mode", "test", "level", "version", "time"]).reset_index(drop=True)

,descriptor,path,date_beg,date_end,time_beg,time_end
0,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:51:38.000,2025-05-04 05:53:08.000
1,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:53:38.000,2025-05-04 05:55:08.000
2,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:55:38.000,2025-05-04 05:57:08.000
3,housekeeping,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-05,2025-05-04 05:57:38.000,2025-05-05 05:59:08.000
4,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 05:53:11.353,2025-05-04 05:53:11.830
5,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 07:37:49.472,2025-05-04 07:37:49.476
6,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-04,2025-05-04 08:03:30.385,2025-05-04 08:03:30.390
7,photon,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2025-05-04,2025-05-05,2025-05-04 08:32:34.299,2025-05-05 08:32:34.430
8,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 10:39:51.392
9,spectrum,/Users/andrewrobbertz/__SOC_CODE__/_data_/padr...,2000-01-01,2025-05-04,2000-01-01 00:00:00.000,2025-05-04 11:50:51.406


## Check for SOLARNET Compliance in the L0 (Stripped) Data

In [28]:
# Create Custome PADRE SOLARNET schema
padre_schema = SOLARNETSchema(schema_layers=[CUSTOM_ATTRS_PATH])

files = []
all_findings = []
for processed_file in result_df["path"]:
    # Validate the first Processed File against the SOLARNET schema
    file_findings = validate_file(
        file_path=processed_file,
        warn_empty_keyword=True,
        warn_no_comment=False,
        warn_data_type=True,
        schema=padre_schema,
    )
    all_findings.extend(file_findings)
    files.extend([processed_file.name] * len(file_findings))

In [29]:
df = pd.DataFrame([files, all_findings]).T
df.columns = ["file", "findings"]

# Group by findings and get unique filenames for each finding
findings_summary = df.groupby('findings')['file'].unique().reset_index()

# Optionally, add a count of files for each finding
findings_summary['file_count'] = findings_summary['file'].apply(len)

# Sort by most common findings first
findings_summary = findings_summary.sort_values('file_count', ascending=True)

findings_summary.head(10)

,findings,file,file_count
11,Observation Header 2: Keyword 'JDREF' not foun...,[padre_meddea_l0test_housekeeping_20250504T055...,4
16,Observation Header 2: Keyword 'TUNIT1' not fou...,[padre_meddea_l0test_housekeeping_20250504T055...,4
15,Observation Header 2: Keyword 'TREFPOS' not fo...,[padre_meddea_l0test_housekeeping_20250504T055...,4
13,Observation Header 2: Keyword 'TCUNI1' not fou...,[padre_meddea_l0test_housekeeping_20250504T055...,4
2,Observation Header 1: Keyword 'TFIELDS' not fo...,[padre_meddea_l0test_housekeeping_20250504T055...,8
21,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T055...,12
20,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T055...,12
19,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T055...,12
18,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T055...,12
17,Observation Header 2: Missing Required Attribu...,[padre_meddea_l0test_housekeeping_20250504T055...,12


In [30]:
pprint(findings_summary["findings"].values)

array(["Observation Header 2: Keyword 'JDREF' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 2: Keyword 'TUNIT1' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 2: Keyword 'TREFPOS' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 2: Keyword 'TCUNI1' not found in the schema. Cannot Validate Data Type.",
       "Observation Header 1: Keyword 'TFIELDS' not found in the schema. Cannot Validate Data Type.",
       'Observation Header 2: Missing Required Attribute: CTYPEia. No pattern match for CTYPEia with pattern CTYPE(?P<i>[1-9])(?P<a>[A-Z])?',
       'Observation Header 2: Missing Required Attribute: CRVALia. No pattern match for CRVALia with pattern CRVAL(?P<i>[1-9])(?P<a>[A-Z])?',
       'Observation Header 2: Missing Required Attribute: CRPIXja. No pattern match for CRPIXja with pattern CRPIX(?P<i>[1-9])(?P<a>[A-Z])?',
       'Observation Header 2: Missing Required Attribute: CNAMEia. N